<a href="https://colab.research.google.com/github/lovishdany/NLP_NCU/blob/main/Chatbot_summarizer1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installing required libraries


In [ ]:
#installing required libraries
!pip install transformers
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.1 MB/s eta 0:00:00


# 2. Building Connection

In [ ]:
import requests
import groq

# Defining Groq API endpoint and secret API key
groq_api_url = "https://api.groq.com/openai/v1/chat/completions"
api_key = "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"

from groq import Groq

client = groq.Groq(api_key=api_key)

# 3. Defining ChatBot Function

In [ ]:
#defining function for chatbot to generate response by using the specific model and given parameters

# chat_history = []  # To store the conversation

def get_chatbot_response(user_input):
    # global chat_history

    # Call the Groq API
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "you are a helpful assistant."},
            {"role": "user", "content": user_input}
        ],
        model="llama3-8b-8192",
        temperature=0.2,
        max_tokens=100,
        frequency_penalty=0.5,
        top_p=0.3,
        stop=None,
        stream=False,
    )
    response = chat_completion.choices[0].message.content

    # Append to chat history
    # chat_history.append({"user": user_input, "chatbot": response})
    return response


# 4. Custom Dataset

In [ ]:
conversations = [
    """User: Hello
Chatbot: Hi there! How can I assist you today?
User: Can you tell me about the weather forecast for today?
Chatbot: Sure! Today’s forecast shows sunny skies with a high of 25°C and a low of 15°C.""",

    """User: I want to go to the market.
Chatbot: Sure, let me know if you need help finding directions.
User: Can you tell me the closest market nearby?
Chatbot: The nearest market is Greenfield Market, which is about 2 km away.
User: Thanks!
Chatbot: You’re welcome! Have a great trip.""",

    """User: I am looking for a good book to read.
Chatbot: What genre do you prefer? Fiction, non-fiction, or something specific?
User: I like mystery novels.
Chatbot: I recommend 'The Silent Patient' by Alex Michaelides. It’s a gripping psychological thriller.
User: That sounds interesting. Thank you!
Chatbot: My pleasure! Happy reading.""",

    """User: Can you help me with a recipe for pasta?
Chatbot: Of course! What kind of pasta are you planning to make?
User: Something simple with tomato sauce.
Chatbot: Here's a simple recipe: Cook your pasta as per the package instructions. In a pan, sauté garlic in olive oil, add tomato puree, season with salt, pepper, and basil, and let it simmer. Toss the pasta in the sauce, and it's ready!
User: That’s perfect. Thanks!
Chatbot: You’re welcome! Enjoy your meal.""",

    """User: I need help setting up my email on my phone.
Chatbot: Sure! Are you using an iPhone or an Android device?
User: I’m using an iPhone.
Chatbot: Go to Settings, then tap on Mail. Select 'Accounts' and choose 'Add Account.' Enter your email credentials and follow the on-screen instructions.
User: Got it. Thanks a lot.
Chatbot: Happy to help! Let me know if you face any issues."""
]

summaries = [
    "The user greeted the chatbot and asked about the weather forecast. The chatbot provided the day’s weather details.",
    "The user expressed a desire to visit a market and asked for the closest one. The chatbot gave a suggestion and wished them a good trip.",
    "The user was looking for a mystery novel recommendation. The chatbot suggested 'The Silent Patient' and wished them happy reading.",
    "The user asked for a simple pasta recipe. The chatbot provided step-by-step instructions for making pasta with tomato sauce.",
    "The user needed help setting up email on their iPhone. The chatbot provided detailed instructions and offered further assistance if needed."
]


# 5. Preparing and pre-processing dataset

In [ ]:
#pre-processing and preparing the dataset for model training

from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, TensorDataset
import torch

# Load the pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Function to tokenize conversations and summaries
def prepare_dataset(conversations, summaries, tokenizer, input_max_len=512, target_max_len=150):
    input_ids = []
    attention_masks = []
    labels = []

    for conversation, summary in zip(conversations, summaries):
        # Tokenize input conversation
        input_enc = tokenizer(
            "summarize: " + conversation,
            truncation=True,
            padding="max_length",
            max_length=input_max_len,
            return_tensors="pt"
        )
        # Tokenize target summary
        target_enc = tokenizer(
            summary,
            truncation=True,
            padding="max_length",
            max_length=target_max_len,
            return_tensors="pt"
        )

        # Append tokenized data to lists
        input_ids.append(input_enc.input_ids.squeeze())
        attention_masks.append(input_enc.attention_mask.squeeze())
        labels.append(target_enc.input_ids.squeeze())

    # Convert lists to tensors and return as TensorDataset
    return TensorDataset(
        torch.stack(input_ids),
        torch.stack(attention_masks),
        torch.stack(labels)
    )


# Prepare dataset using the function
dataset = prepare_dataset(conversations, summaries, tokenizer)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Iterating over the dataloader
for batch in dataloader:
    input_ids, attention_masks, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Masks:", attention_masks)
    print("Labels:", labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Input IDs: tensor([[    0, 18581,  3916,  ...,     1,     1,     1],
        [    0, 18581,  3916,  ...,     1,     1,     1]])
Attention Masks: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Labels: tensor([[    0,   133,  3018,  2327,    10,  4724,     7,   825,    10,   210,
             8,   553,    13,     5,  8099,    65,     4,    20,  7359, 12749,
           851,    10, 10791,     8, 14594,   106,    10,   205,  1805,     4,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,   

# 6. Model Training

In [ ]:
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Fine-tuning the model
model.train()
epochs = 12
for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_tokens = 0

    for batch in dataloader:
        # Unpacking the batch (each batch contains input_ids, attention_masks, and labels)
        input_ids, attention_masks, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
        input_ids=input_ids,
        attention_mask=attention_masks,
        labels=labels
        )

        # Loss calculation
        loss = outputs.loss
        total_loss += loss.item()

        # Accuracy calculation
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)


        mask = labels != tokenizer.pad_token_id
        correct = (predictions == labels) & mask
        total_correct += correct.sum().item()
        total_tokens += mask.sum().item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Calculating average loss and accuracy
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_tokens * 100

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Saving the fine-tuned model
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")
print("Model fine-tuned and saved!")


Epoch 1/12, Loss: 9.5113, Accuracy: 56.12%
Epoch 2/12, Loss: 8.2956, Accuracy: 64.03%
Epoch 3/12, Loss: 7.3758, Accuracy: 74.10%
Epoch 4/12, Loss: 6.3992, Accuracy: 82.01%
Epoch 5/12, Loss: 5.1207, Accuracy: 84.17%
Epoch 6/12, Loss: 3.8658, Accuracy: 87.05%
Epoch 7/12, Loss: 2.8785, Accuracy: 87.77%
Epoch 9/12, Loss: 1.7573, Accuracy: 92.09%
Epoch 10/12, Loss: 1.4426, Accuracy: 94.96%
Epoch 11/12, Loss: 1.1971, Accuracy: 92.81%
Epoch 12/12, Loss: 1.0359, Accuracy: 95.68%


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model fine-tuned and saved!


# 7. Summarization Function

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model
fine_tuned_model_path = "/content/fine_tuned_model"
model = BartForConditionalGeneration.from_pretrained(fine_tuned_model_path)
tokenizer = BartTokenizer.from_pretrained(fine_tuned_model_path)

# Function to summarize chat history using the fine-tuned model
def summarize_chat_finetuned(chat_history):

    # Combining chat history into a single text
    chat_text = "\n".join([f"User: {item['user']}\nChatbot: {item['chatbot']}" for item in chat_history])

    # Preparing the input for the model
    input_text = "summarize: " + chat_text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    outputs = model.generate(inputs, max_length=70, min_length=30, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


# 8. Final Execution

In [ ]:
chat_history = []  # To store the conversation

def chat_with_bot():
    global chat_history
    print("Chatbot: Hello! How can I assist you today? Type 'exit' to end chat.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nChatbot: Goodbye!")

            # Summarize chat at the end
            summary = summarize_chat_finetuned(chat_history)
            print("\nChat Summary:")
            print(summary)
            break

        response = get_chatbot_response(user_input)
        # Append to chat history
        chat_history.append({"user": user_input, "chatbot": response})

        print(f"\nChatbot: {response}")

# Calling function for chatting with the bot
chat_with_bot()

Chatbot: Hello! How can I assist you today? Type 'exit' to end chat.

You: hey

Chatbot: Hey! How can I help you today? Do you have a specific question, need assistance with something, or just want to chat? I'm here to help!

You: I want to go for movies

Chatbot: Going to the movies sounds like a fun plan! I'd be happy to help you with that.

Can you please tell me a bit more about what you're looking for? For example:

1. What type of movies are you in the mood for (action, comedy, romance, etc.)?
2. Are you open to watching a new release or do you have a specific movie in mind?
3. Do you have a preferred cinema or location in mind?
4. Are you planning to go

You: bye

Chatbot: Goodbye!

Chat Summary:
The user expressed a desire to go to the movies and asked about going to the movies. The chatbot offered to help the user find out what type of movies they were looking for and where they planned to go. The user expressed a desire to go to the movies and asked
